In [1]:
from collections import deque
from typing import List, Tuple, Optional, Dict

State = Tuple[Tuple[int, ...], ...]      # tuple of towers; each tower is a tuple of piece IDs, bottom -> top
Move  = Tuple[int, int]                  # (from_tower, to_tower)

def solve_skyscrapers(start: State,
                      goal: State,
                      *,
                      return_states: bool = False,
                      one_indexed: bool = False) -> Dict[str, object]:
    """
    Find an optimal (minimum-move) sequence of moves to transform `start` into `goal`.

    Representation:
      - `start` and `goal` are tuples of towers; each tower is a tuple of piece IDs (ints) from bottom -> top.
      - Only the TOP piece of a tower (last element) may be moved onto the TOP of another tower.

    Returns dict with:
      - "moves": List[Move]        # optimal move list (from_tower, to_tower)
      - "num_moves": int           # length of the optimal path
      - "states": Optional[List[State]]  # states along the optimal path (if return_states=True)

    Args:
      return_states: also return the sequence of states along the optimal path (including start and goal).
      one_indexed:   if True, moves are returned 1-indexed (towers numbered 1..N) for easier reading.

    Raises:
      ValueError if `start` and `goal` do not contain the same multiset of piece IDs.
    """
    # normalize to tuples of tuples
    start = tuple(tuple(t) for t in start)
    goal  = tuple(tuple(t) for t in goal)

    # sanity: same pieces in both states
    def multiset(state: State):
        out = []
        for t in state: out.extend(t)
        out.sort()
        return tuple(out)
    if multiset(start) != multiset(goal):
        raise ValueError("`start` and `goal` must contain the same multiset of piece IDs.")

    if start == goal:
        return {"moves": [], "num_moves": 0, "states": [start] if return_states else None}

    # BFS
    q = deque([start])
    parent: Dict[State, Optional[State]] = {start: None}
    via_move: Dict[State, Move] = {}

    def neighbors(s: State):
        n = len(s)
        for i, stack in enumerate(s):
            if not stack: 
                continue
            top = stack[-1]
            for j in range(n):
                if i == j: 
                    continue
                # build next state
                new_towers = [list(t) for t in s]
                new_towers[i].pop()
                new_towers[j].append(top)
                yield tuple(tuple(t) for t in new_towers), (i, j)

    # run BFS
    while q:
        cur = q.popleft()
        for nxt, mv in neighbors(cur):
            if nxt in parent:
                continue
            parent[nxt] = cur
            via_move[nxt] = mv
            if nxt == goal:
                # reconstruct
                rev_moves: List[Move] = []
                path_states: List[State] = [nxt]
                walk = nxt
                while parent[walk] is not None:
                    rev_moves.append(via_move[walk])
                    walk = parent[walk]
                    path_states.append(walk)
                rev_moves.reverse()
                path_states.reverse()
                moves = rev_moves
                if one_indexed:
                    moves = [(a+1, b+1) for a, b in moves]
                return {
                    "moves": moves,
                    "num_moves": len(moves),
                    "states": path_states if return_states else None
                }
            q.append(nxt)

    # Should be reachable under these rules.
    return {"moves": None, "num_moves": None, "states": None}


# Optional helpers:

def next_optimal_move(start: State, goal: State, *, one_indexed: bool = False) -> Optional[Move]:
    """Return the first move of an optimal path from `start` to `goal` (or None if already solved)."""
    res = solve_skyscrapers(start, goal, return_states=False, one_indexed=one_indexed)
    if res["num_moves"] in (None, 0):
        return None
    return res["moves"][0]

def make_target(num_towers: int, num_pieces: int, target_tower_idx: int) -> State:
    """Build a goal state where pieces 0..num_pieces-1 are stacked (bottom->top) on `target_tower_idx`."""
    target = [tuple() for _ in range(num_towers)]
    target[target_tower_idx] = tuple(range(num_pieces))
    return tuple(target)


In [2]:
def format_moves(moves, *, one_indexed: bool = True) -> str:
    """
    Convert a list of (from_tower, to_tower) moves into:
    '1. tower 2 to tower 1' lines.

    one_indexed=True prints towers as 1..N; set False for 0..N-1.
    """
    lines = []
    for i, (a, b) in enumerate(moves, 1):
        if one_indexed:
            a += 1; b += 1
        lines.append(f"{i}. tower {a} to tower {b}")
    return "\n".join(lines)


In [23]:
start = ((3,1), (2,), (0,4))             # bottom->top per tower
goal  = make_target(3, 5, target_tower_idx=0)

res = solve_skyscrapers(start, goal, return_states=False, one_indexed=True)
print(res["num_moves"])     # optimal move count
print(res["moves"])         # e.g. [(3,2), (1,3), ...] with 1-based tower indices

# Example 2: just get the next optimal move (0-indexed)
hint = next_optimal_move(start, goal)
print(hint)  # (from_tower, to_tower)

10
[(1, 2), (1, 2), (3, 2), (3, 1), (2, 3), (2, 3), (2, 1), (2, 1), (3, 1), (3, 1)]
(0, 1)


In [24]:
res = solve_skyscrapers(start, goal)      # returns 0-indexed moves
print(format_moves(res["moves"], one_indexed=True))

1. tower 1 to tower 2
2. tower 1 to tower 2
3. tower 3 to tower 2
4. tower 3 to tower 1
5. tower 2 to tower 3
6. tower 2 to tower 3
7. tower 2 to tower 1
8. tower 2 to tower 1
9. tower 3 to tower 1
10. tower 3 to tower 1
